In [1]:
import torch

from ClusterDataset import ClusterDataset

from sklearn.model_selection import train_test_split

In [5]:
# Load the dataset
input_folder = "/eos/user/c/czeh/histo/"
output_folder = "/eos/user/c/czeh/graph_data"
dataset = ClusterDataset(output_folder, input_folder)

idx = list(range(dataset.len()))

In [8]:
dataset.processed_file_names

['/eos/user/c/czeh/graph_data/processed/data_0.pt',
 '/eos/user/c/czeh/graph_data/processed/data_1.pt',
 '/eos/user/c/czeh/graph_data/processed/data_2.pt',
 '/eos/user/c/czeh/graph_data/processed/data_3.pt',
 '/eos/user/c/czeh/graph_data/processed/data_4.pt',
 '/eos/user/c/czeh/graph_data/processed/data_5.pt',
 '/eos/user/c/czeh/graph_data/processed/data_6.pt',
 '/eos/user/c/czeh/graph_data/processed/data_7.pt',
 '/eos/user/c/czeh/graph_data/processed/data_8.pt',
 '/eos/user/c/czeh/graph_data/processed/data_9.pt',
 '/eos/user/c/czeh/graph_data/processed/pre_filter.pt',
 '/eos/user/c/czeh/graph_data/processed/pre_transform.pt']